# Assigns standardized locations to entities in the KG
**[Work in progress]**

This notebook standardizes location information for ingestion into a Knowledge Graph.

Author: Peter Rose (pwrose@ucsd.edu), Braden Riggs 

In [1]:
import os
import pandas as pd
from pathlib import Path
import time
import unidecode
import difflib

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/import


### Import data sources that have an `origLocation` property

In [4]:
df1 = pd.read_csv(NEO4J_IMPORT / '01b-Nextstrain.csv', dtype='str', usecols=['origLocation'])
df2 = pd.read_csv(NEO4J_IMPORT / '01d-CNCBStrain.csv', dtype='str', usecols=['origLocation'])
df3 = pd.read_csv(NEO4J_IMPORT / '02a-JHUCasesGlobal.csv', dtype='str', usecols=['origLocation'])
df4 = pd.read_csv(NEO4J_IMPORT / '02b-CDSCases.csv', dtype='str', usecols=['origLocation'])
df5 = pd.read_csv(NEO4J_IMPORT / '02d-GOBMXCasesAdmin1.csv', dtype='str', usecols=['origLocation'])
df6 = pd.read_csv(NEO4J_IMPORT / '02d-GOBMXCasesAdmin2.csv', dtype='str', usecols=['origLocation'])

df = pd.concat([df1, df2, df3, df4, df5, df6])

In [5]:
df.drop_duplicates(inplace=True)
df.fillna('', inplace=True)

In [6]:
df.shape

(8770, 1)

In [7]:
df.head()

,origLocation
0,"China,Guangdong,Zhuhai"
1,"Taiwan,Taipei"
2,"China,Hubei,Wuhan"
7,"Thailand,Bangkok"
8,Thailand


In [8]:
df[df['origLocation'].str.contains('D.C.')].head()

,origLocation
274903,"United States,Washington, D.C."
275124,"United States,Washington, D.C.,District of Col..."


### Load custom GeoName mappings

In [9]:
ref = pd.read_csv("../../reference_data/GeoNameMapping.csv", usecols=['origName', 'geoName'], comment='#', dtype='str')

In [10]:
ref.head()

,origName,geoName
0,Burma,Myanmar
1,Cape Verde,Cabo Verde
2,Congo (Kinshasa),Democratic Republic of the Congo
3,Congo (Brazzaville),Republic of the Congo
4,Congo-Brazzaville,Republic of the Congo


In [11]:
name_list = ref[['origName', 'geoName']].values.tolist()

In [12]:
def fix_locations(text):
    for loc in name_list:
        if text.startswith(loc[0]):
            # full length match or match at a comma
            if len(text) == len(loc[0]) or text[len(loc[0])] == ',':
                text = text.replace(loc[0], loc[1])
    return text

### Apply custom GeoName mappings

In [13]:
df['geoLocation'] = df['origLocation'].apply(lambda x: fix_locations(x))

In [14]:
df.query('geoLocation != origLocation').head()

,origLocation,geoLocation
10,"USA,California","United States,California"
75,"USA,California,Orange County CA","United States,California,Orange County"
126,"USA,Arizona,Phoenix","United States,Arizona,Phoenix"
143,"USA,California,Los Angeles County","United States,California,Los Angeles County"
150,"USA,Illinois","United States,Illinois"


### Match Cruiseships

In [15]:
cruiseships = df[df['geoLocation'].str.startswith('CruiseShip,')].copy()

In [16]:
cruiseships.head(10)

,origLocation,geoLocation
13353,"Italy,Cruise ship","CruiseShip,Cruise ship"
45783,"United States,Cruise_Ship_1,California","CruiseShip,Cruise_Ship_1"
45789,"United States,Cruise_Ship_2,California","CruiseShip,Cruise_Ship_2"
864,Diamond Princess,"CruiseShip,Diamond Princess"
4103,"Canada,Grand Princess","CruiseShip,Grand Princess"
6003,"Canada,Diamond Princess","CruiseShip,Diamond Princess"
7590,MS Zaandam,"CruiseShip,MS Zaandam"


In [17]:
cruiseships = cruiseships[['origLocation', 'geoLocation']]
cruiseships.to_csv(NEO4J_IMPORT / "10a-GeoLinkCruiseShip.csv", index=False)

##### Remove Cruiseship data from data frame

In [18]:
df = df[~df['geoLocation'].str.startswith('CruiseShip,')].copy()

In [19]:
df.shape

(8763, 2)

In [20]:
def remove_special_characters(text):
    text = str(text)
    text = text.lower()
    # transliterate umlauts
    text = text.replace('ü', 'ue') 
    text = text.replace('ä', 'ae') 
    text = text.replace('ö', 'oe')
    # transliterate Unicode string into the closest possible ASCII representation
    text = unidecode.unidecode(text)
    text = text.replace("-"," ")
    text = text.replace("_"," ")
    text = text.replace(","," ")
    text = text.replace("."," ")
    text = text.replace('\d+', '')
    text = text.strip()

    return text

In [21]:
def remove_affix(text):
    text = str(text)
    text = text.lower()
    # try match without prefix or suffix
    text = text.replace("region", "")
    text = text.replace("oblast", "") # regions in Russia
    text = text.replace("al ", "") # regions in Saudi Arabia
    text = text.replace("prefecture", "") # regions in Japan
    text = text.replace("province", "") # regions in China
    text = text.replace("district", "")
    text = text.replace("canton", "") # regions in Switzerland
    text = text.replace("municipality", "")
    text = text.replace("municipio", "")  # regions in Mexico
    text = text.replace("metropolitan", "")
    text = text.replace("voivodeship", "")
    text = text.replace("state", "")
    text = text.replace('\d+', '')
    text = text.strip()
    return text

In [22]:
def remove_affix2(text):
    text = str(text)
    text = text.lower()
    # try match without prefix or suffix
    text = text.replace("city", "")
    text = text.replace("region", "")
    text = text.replace("county", "")
    text = text.replace("oblast", "") # regions in Russia
    text = text.replace("al ", "") # regions in Saudi Arabia
    text = text.replace("prefecture", "") # regions in Japan
    text = text.replace("province", "") # regions in China
    text = text.replace("district", "")
    text = text.replace("parish", "")
    text = text.replace("gemeente", "") # regions in the Netherlands
    text = text.replace("wahlkreis", "") # regions in Switzerland
    text = text.replace("canton", "") # regions in Switzerland
    text = text.replace("municipality", "")
    text = text.replace("municipio", "") # regions in Mexico
    text = text.replace("metropolitan", "")
    text = text.replace("voivodeship", "")
    text = text.replace("novads", "")
    text = text.replace("state", "")
    text = text.replace("st ", "")
    text = text.replace('\d+', '')
    text = text.strip()
    return text

In [23]:
def get_close_match(x, name):
    z = difflib.get_close_matches(x, name, n=1, cutoff=0.9)
    if z:
        return z[0]
    else:
        return ''

In [24]:
df.shape

(8763, 2)

In [25]:
df['locationLevels'] = df['geoLocation'].str.count(',')
df[['loc0', 'loc1', 'loc2', 'loc3']] = df['geoLocation'].str.split(',', n=3, expand=True)
df.fillna('', inplace=True)

## Match Countries

In [26]:
countries = pd.DataFrame(df['loc0'].copy())
countries.drop_duplicates(inplace=True)
countries.dropna(inplace=True)
countries.columns=['origName']
countries['canonicalName'] = countries.apply(lambda x: remove_special_characters(x['origName']), axis=1)

In [27]:
countries.shape

(222, 2)

In [28]:
countries.head()

,origName,canonicalName
0,China,china
1,Taiwan,taiwan
7,Thailand,thailand
10,United States,united states
15,Canada,canada


In [29]:
geo_countries = pd.read_csv(NEO4J_IMPORT / "00e-GeoNamesCountry.csv", dtype='str', usecols=['name'])

In [30]:
geo_countries['canonicalName'] = geo_countries.apply(lambda x: remove_special_characters(x['name']), axis=1)

In [31]:
countries = countries.merge(geo_countries, on='canonicalName', how='left')
countries.fillna('', inplace=True)

### Exact country matches

In [32]:
country_exact_match = countries.query("name != ''").copy()

In [33]:
country_exact_match.shape

(222, 3)

In [34]:
country_no_match = countries.query("name == ''").copy()

In [35]:
country_no_match.head(10)

,origName,canonicalName,name


### Match Admin1

In [36]:
admin1 = pd.DataFrame(df['loc1'].copy())
admin1.drop_duplicates(inplace=True)
admin1.dropna(inplace=True)
admin1.columns=['origName']
admin1['canonicalName'] = admin1.apply(lambda x: remove_special_characters(x['origName']), axis=1)

In [37]:
geo_admin1 = pd.read_csv(NEO4J_IMPORT / "00f-GeoNamesAdmin1.csv", dtype='str', usecols=['name'])

In [38]:
geo_admin1['canonicalName'] = geo_admin1.apply(lambda x: remove_special_characters(x['name']), axis=1)

In [39]:
admin1 = admin1.merge(geo_admin1, on='canonicalName', how='left')
admin1.fillna('', inplace=True)

In [40]:
admin1_exact_match = admin1.query("name != ''").copy()
admin1_exact_match.drop_duplicates(inplace=True)

In [41]:
admin1_exact_match.shape

(986, 3)

In [42]:
admin1_exact_match.head(10)

,origName,canonicalName,name
0,Guangdong,guangdong,Guangdong
1,Taipei,taipei,Taipei
2,Hubei,hubei,Hubei
3,Bangkok,bangkok,Bangkok
5,California,california,California
6,Ontario,ontario,Ontario
7,Shanghai,shanghai,Shanghai
9,Hokkaido,hokkaido,Hokkaido
11,Kuala Lumpur,kuala lumpur,Kuala Lumpur
12,New South Wales,new south wales,New South Wales


##### Remaining locations that don't match a GeoName location

In [43]:
admin1 = admin1.query("name == ''").copy()
admin1.drop('name', axis=1, inplace=True)

In [44]:
admin1.shape

(3335, 2)

In [45]:
admin1['canonicalNameNoAffix'] = admin1.apply(lambda x: remove_affix(x['canonicalName']), axis=1)

In [46]:
geo_admin1['canonicalNameNoAffix'] = geo_admin1.apply(lambda x: remove_affix(x['canonicalName']), axis=1)
admin1 = admin1.merge(geo_admin1[['name', 'canonicalNameNoAffix']], on='canonicalNameNoAffix', how='left')
admin1.fillna('', inplace=True)

In [47]:
admin1_no_affix_match = admin1.query("name != ''")

In [48]:
admin1_no_affix_match.shape

(265, 4)

In [49]:
admin1_no_affix_match.head()

,origName,canonicalName,canonicalNameNoAffix,name
26,Osun State,osun state,osun,Osun
27,Kwara State,kwara state,kwara,Kwara
53,Chisinau,chisinau,chisinau,Chisinau Municipality
55,Ogun State,ogun state,ogun,Ogun
57,Oyo State,oyo state,oyo,Oyo


In [50]:
admin1_no_match = admin1.query("name == ''").copy()
admin1_no_match.query("origName != ''", inplace=True)

In [51]:
admin1_no_match.shape

(3076, 4)

In [52]:
admin1_no_match.head()

,origName,canonicalName,canonicalNameNoAffix,name
1,Kanto,kanto,kanto,
2,Hokurikushinsyu,hokurikushinsyu,hokurikushinsyu,
3,Hong Kong,hong kong,hong kong,
4,Kaohsiung,kaohsiung,kaohsiung,
5,Samut Prakarn,samut prakarn,samut prakarn,


### Match Admin2

In [53]:
admin2 = pd.DataFrame(df['loc2'].copy())
admin2.columns=['origName']
admin2.query("origName != ''", inplace=True)
admin2.head()

,origName
0,Zhuhai
2,Wuhan
36,Huizhou
37,Sydney
39,Clayton AU


In [54]:
a1_no_match = pd.DataFrame(admin1_no_match['origName']).copy()

In [55]:
a1_no_match.shape

(3076, 1)

In [56]:
a1_no_match.head()

,origName
1,Kanto
2,Hokurikushinsyu
3,Hong Kong
4,Kaohsiung
5,Samut Prakarn


In [57]:
# carry over regions from location level 1 that might be Admin2s or Cities
admin2 = pd.concat([admin2,a1_no_match])
admin2.drop_duplicates(inplace=True)
admin2.dropna(inplace=True)
admin2.columns=['origName']
admin2['canonicalName'] = admin2.apply(lambda x: remove_special_characters(x['origName']), axis=1)

In [58]:
geo_admin2 = pd.read_csv(NEO4J_IMPORT / "00g-GeoNamesAdmin2.csv", dtype='str', usecols=['name'])

In [59]:
geo_admin2['canonicalName'] = geo_admin2.apply(lambda x: remove_special_characters(x['name']), axis=1)

In [60]:
admin2 = admin2.merge(geo_admin2, on='canonicalName', how='left')
admin2.fillna('', inplace=True)

In [61]:
admin2_exact_match = admin2.query("name != ''").copy()
admin2_exact_match.drop_duplicates(inplace=True)

In [62]:
admin2_exact_match.shape

(4229, 3)

In [63]:
admin2_exact_match.head()

,origName,canonicalName,name
6,Melbourne,melbourne,Melbourne
7,Orange County,orange county,Orange County
15,Paris,paris,Paris
17,Brisbane,brisbane,Brisbane
23,Los Angeles County,los angeles county,Los Angeles County


##### Remaining locations that don't match a GeoName location

In [64]:
admin2_no_match = admin2.query("name == ''").copy()

In [65]:
admin2_no_match.shape

(1550, 3)

### Match Cities

In [66]:
city = pd.DataFrame(df['loc3'].copy())
city.columns=['origName']
city.query("origName != ''", inplace=True)
city.head(100)

,origName
1589,Queens
1825,Manhattan
466,Sheffield
2339,Manhattan
3047,Brooklyn
3587,Queens
9663,Havlickuv Brod
13320,St Anton am Arlberg
35136,Bilbao
38148,Whitehall


In [67]:
a2_no_match = pd.DataFrame(admin2_no_match['origName'])

In [68]:
# carry over regions from location level 1 that might be Admin2s or Cities
city = pd.concat([a2_no_match, city])
city.drop_duplicates(inplace=True)
city.dropna(inplace=True)
city.columns=['origName']
city['canonicalName'] = city.apply(lambda x: remove_special_characters(x['origName']), axis=1)

In [69]:
geo_city = pd.read_csv(NEO4J_IMPORT / "00h-GeoNamesCity.csv", dtype='str', usecols=['name'])

In [70]:
geo_city['canonicalName'] = geo_city.apply(lambda x: remove_special_characters(x['name']), axis=1)

In [71]:
city = city.merge(geo_city, on='canonicalName', how='left')
city.fillna('', inplace=True)

In [72]:
city_exact_match = city.query("name != ''").copy()
city_exact_match.drop_duplicates(inplace=True)

In [73]:
city_exact_match.shape

(787, 3)

In [74]:
city_exact_match.head()

,origName,canonicalName,name
0,Zhuhai,zhuhai,Zhuhai
1,Wuhan,wuhan,Wuhan
2,Huizhou,huizhou,Huizhou
3,Sydney,sydney,Sydney
6,Jönköping,joenkoeping,Joenkoeping


In [75]:
no_match = city.query("name == ''").copy()

In [76]:
no_match.shape

(773, 3)

In [77]:
no_match.head()

,origName,canonicalName,name
5,Clayton AU,clayton au,
15,Gyeonggi,gyeonggi,
17,Gomez Palacios,gomez palacios,
18,Amelia Denis de Icaza,amelia denis de icaza,
26,Belisario Frías,belisario frias,


### Match other divisions

In [78]:
divisions = no_match['origName']

In [79]:
divisions.drop_duplicates(inplace=True)
divisions.dropna(inplace=True)
divisions = divisions[divisions != '']
divisions.shape

(773,)

In [80]:
divisions = pd.DataFrame(divisions)
divisions.columns=['origName']

In [81]:
divisions['canonicalName'] = divisions.apply(lambda x: remove_special_characters(x['origName']), axis=1)

In [82]:
divisions.head()

,origName,canonicalName
5,Clayton AU,clayton au
15,Gyeonggi,gyeonggi
17,Gomez Palacios,gomez palacios
18,Amelia Denis de Icaza,amelia denis de icaza
26,Belisario Frías,belisario frias


In [83]:
geo_admin1 = pd.read_csv(NEO4J_IMPORT / "00f-GeoNamesAdmin1.csv", dtype='str', usecols=['name'])
geo_admin2 = pd.read_csv(NEO4J_IMPORT / "00g-GeoNamesAdmin2.csv", dtype='str', usecols=['name'])
geo_city = pd.read_csv(NEO4J_IMPORT / "00h-GeoNamesCity.csv", dtype='str', usecols=['name'])

In [84]:
geo_divisions = pd.concat([geo_admin1, geo_admin2, geo_city])

In [85]:
geo_divisions.drop_duplicates(inplace=True)

In [86]:
geo_divisions.shape

(154242, 1)

In [87]:
geo_divisions.dropna(inplace=True)
geo_divisions.columns=['name']
geo_divisions['canonicalName'] = geo_divisions.apply(lambda x: remove_special_characters(x['name']), axis=1)

In [88]:
geo_divisions.head()

,name,canonicalName
0,Sant Julia de Loria,sant julia de loria
1,Ordino,ordino
2,La Massana,la massana
3,Encamp,encamp
4,Canillo,canillo


In [89]:
divisions['canonicalNameNoAffix'] = divisions.apply(lambda x: remove_affix2(x['canonicalName']), axis=1)

In [90]:
geo_divisions['canonicalNameNoAffix'] = geo_divisions.apply(lambda x: remove_affix2(x['canonicalName']), axis=1)
divisions = divisions.merge(geo_divisions[['name', 'canonicalNameNoAffix']], on='canonicalNameNoAffix', how='left')
divisions.fillna('', inplace=True)

In [91]:
divisions.head(1000)

,origName,canonicalName,canonicalNameNoAffix,name
0,Clayton AU,clayton au,clayton au,
1,Gyeonggi,gyeonggi,gyeonggi,
2,Gomez Palacios,gomez palacios,gomez palacios,
3,Amelia Denis de Icaza,amelia denis de icaza,amelia denis de icaza,
4,Belisario Frías,belisario frias,belisario frias,
5,Vitoria-Gasteiz,vitoria gasteiz,vitoria gasteiz,
6,Omar Torrijos,omar torrijos,omar torrijos,
7,Ernesto Córdoba,ernesto cordoba,ernesto cordoba,
8,Donostia-San Sebastian,donostia san sebastian,donostia san sebastian,
9,Arnulfo Arias,arnulfo arias,arnulfo arias,


In [92]:
divisions_exact_match = divisions.query("name != ''")

In [93]:
divisions_exact_match.shape

(312, 4)

In [94]:
divisions_exact_match.head()

,origName,canonicalName,canonicalNameNoAffix,name
10,Heinsberg District,heinsberg district,heinsberg,Heinsberg
12,Aichi,aichi,aichi,Aichi
29,eThekwini,ethekwini,ethekwini,eThekwini Metropolitan Municipality
37,Mirpur,mirpur,mirpur,Mirpur District
39,Zululand,zululand,zululand,Zululand District Municipality


### Collect all location matches

In [95]:
matches = pd.concat([country_exact_match, admin1_exact_match, admin1_no_affix_match, admin2_exact_match, city_exact_match, divisions_exact_match])

In [96]:
matches = matches[['origName', 'name']]

In [97]:
matches.shape

(6801, 2)

In [98]:
matches.tail()

,origName,name
801,Västernorrland County,Vaesternorrland
802,Jämtland County,Jaemtland
813,Buri Ram Province,Buri Ram
814,Suphan Buri Province,Suphan Buri
819,Mykolaiv Oblast,Mykolaiv


In [99]:
matches.query("origName == 'Zhuhai City'")

,origName,name
72,Zhuhai City,Zhuhai


In [100]:
name_list = matches[['origName', 'name']].values.tolist()
name_dict = {name: val for name, val in name_list}

In [101]:
name_dict.get('Zhuhai City')

'Zhuhai'

#### Assign GeoNames for Countries

In [102]:
country_list = country_exact_match[['origName', 'name']].values.tolist()
country_dict = {name: val for name, val in country_list}

In [103]:
df['geoName0'] = df['loc0'].apply(lambda x: country_dict.get(x, ''))

#### Assign GeoNames for Admin1 divisions

In [104]:
admin1_match = pd.concat([admin1_exact_match, admin1_no_affix_match])
admin1_list = admin1_match[['origName', 'name']].values.tolist()
admin1_dict = {name: val for name, val in admin1_list}

In [105]:
admin1_dict.get("United States,New York")

In [106]:
df['geoName1'] = df['loc1'].apply(lambda x: admin1_dict.get(x, ''))

#### Assign GeoNames for Admin2 divisions

In [107]:
admin2_list = admin2_exact_match[['origName', 'name']].values.tolist()
admin2_dict = {name: val for name, val in admin2_list}

In [108]:
df['geoName2'] = df['loc2'].apply(lambda x: admin2_dict.get(x, ''))

#### Assign GeoNames for Cities

In [109]:
city_list = city_exact_match[['origName', 'name']].values.tolist()
city_dict = {name: val for name, val in city_list}

In [110]:
df['geoName3'] = df['loc3'].apply(lambda x: city_dict.get(x, ''))

In [111]:
df.query("origLocation == 'United States,New York'").head()

,origLocation,geoLocation,locationLevels,loc0,loc1,loc2,loc3,geoName0,geoName1,geoName2,geoName3
455,"United States,New York","United States,New York",1,United States,New York,,,United States,New York,,


#### Assign best match for any unassigned locations

In [112]:
def get_name(loc, geoName):
    if geoName == '':
        return name_dict.get(loc, '')
    else:
        return geoName

In [113]:
df['geoName0'] = df.apply(lambda x: get_name(x['loc0'], x['geoName0']), axis=1)
df['geoName1'] = df.apply(lambda x: get_name(x['loc1'], x['geoName1']), axis=1)
df['geoName2'] = df.apply(lambda x: get_name(x['loc2'], x['geoName2']), axis=1)
df['geoName3'] = df.apply(lambda x: get_name(x['loc3'], x['geoName3']), axis=1)

In [114]:
df.head()

,origLocation,geoLocation,locationLevels,loc0,loc1,loc2,loc3,geoName0,geoName1,geoName2,geoName3
0,"China,Guangdong,Zhuhai","China,Guangdong,Zhuhai",2,China,Guangdong,Zhuhai,,China,Guangdong,Zhuhai,
1,"Taiwan,Taipei","Taiwan,Taipei",1,Taiwan,Taipei,,,Taiwan,Taipei,,
2,"China,Hubei,Wuhan","China,Hubei,Wuhan",2,China,Hubei,Wuhan,,China,Hubei,Wuhan,
7,"Thailand,Bangkok","Thailand,Bangkok",1,Thailand,Bangkok,,,Thailand,Bangkok,,
8,Thailand,Thailand,0,Thailand,,,,Thailand,,,


In [115]:
df[(df['geoName1'] == '') & (df['geoName2'] != '')]

,origLocation,geoLocation,locationLevels,loc0,loc1,loc2,loc3,geoName0,geoName1,geoName2,geoName3
47,"Sweden,Småland,Jönköping","Sweden,Småland,Jönköping",2,Sweden,Småland,Jönköping,,Sweden,,Joenkoeping,
799,"Bahrain,Capital Governorate,Manama","Bahrain,Capital Governorate,Manama",2,Bahrain,Capital Governorate,Manama,,Bahrain,,Manama,
834,"Italy,Lazio,Rome","Italy,Lazio,Rome",2,Italy,Lazio,Rome,,Italy,,Rome,
1040,"Colombia,Departamento de Amazonas,Leticia","Colombia,Departamento de Amazonas,Leticia",2,Colombia,Departamento de Amazonas,Leticia,,Colombia,,Leticia,
1081,"North Macedonia,Northeastern Region,Kumanovo","North Macedonia,Northeastern Region,Kumanovo",2,North Macedonia,Northeastern Region,Kumanovo,,North Macedonia,,Kumanovo,
1426,"Vietnam,Red River Delta,Hanoi","Vietnam,Red River Delta,Hanoi",2,Vietnam,Red River Delta,Hanoi,,Vietnam,,Hanoi,
1932,"Costa Rica,Cartago CR,La Union","Costa Rica,Cartago CR,La Union",2,Costa Rica,Cartago CR,La Union,,Costa Rica,,La Union,
2035,"Poland,Malopolskie,Kraków","Poland,Malopolskie,Kraków",2,Poland,Malopolskie,Kraków,,Poland,,Krakow,
2462,"Czech Republic,Hranice Na Moravě,Prostějov","Czechia,Hranice Na Moravě,Prostějov",2,Czechia,Hranice Na Moravě,Prostějov,,Czechia,,Prostejov,
2612,"Oman,Dakhiliyah,Nizwa","Oman,Dakhiliyah,Nizwa",2,Oman,Dakhiliyah,Nizwa,,Oman,,Nizwa,


### Left align all geoNames

In [116]:
df[['geoName1','geoName2']] = df[['geoName1','geoName2']].apply(lambda x: x if x[0] != '' else [x[1],x[0]], axis=1)
df[['geoName2','geoName3']] = df[['geoName2','geoName3']].apply(lambda x: x if x[0] != '' else [x[1],x[0]], axis=1)

In [117]:
def calc_location_levels(row):
    level = 3
    if row['geoName3'] == '':
        level = 2
    if row['geoName2'] == '':
        level = 1
    if row['geoName1'] == '':
        level = 0
    return level

In [118]:
df['locationLevels'] = df.apply(lambda x: calc_location_levels(x), axis=1)

In [119]:
df.query("origLocation == 'North Macedonia,Pelagonia,Prilep'")

,origLocation,geoLocation,locationLevels,loc0,loc1,loc2,loc3,geoName0,geoName1,geoName2,geoName3
3776,"North Macedonia,Pelagonia,Prilep","North Macedonia,Pelagonia,Prilep",1,North Macedonia,Pelagonia,Prilep,,North Macedonia,Prilep,,


In [120]:
df[(df['geoName1'] == '') & (df['geoName2'] != '')]

,origLocation,geoLocation,locationLevels,loc0,loc1,loc2,loc3,geoName0,geoName1,geoName2,geoName3


Test

In [121]:
df.query("origLocation == 'Austria,St. Anton'")

,origLocation,geoLocation,locationLevels,loc0,loc1,loc2,loc3,geoName0,geoName1,geoName2,geoName3
13320,"Austria,St. Anton","Austria,Tyrol,Politischer Bezirk Landeck,St An...",3,Austria,Tyrol,Politischer Bezirk Landeck,St Anton am Arlberg,Austria,Tyrol,Politischer Bezirk Landeck,St Anton am Arlberg


In [122]:
df.query("geoName1 == 'District of Columbia'").head()

,origLocation,geoLocation,locationLevels,loc0,loc1,loc2,loc3,geoName0,geoName1,geoName2,geoName3
254,"USA,Washington DC","United States,District of Columbia",1,United States,District of Columbia,,,United States,District of Columbia,,
5970,"United States,District of Columbia","United States,District of Columbia",1,United States,District of Columbia,,,United States,District of Columbia,,
6044,"United States,DC","United States,District of Columbia",1,United States,District of Columbia,,,United States,District of Columbia,,
22067,"United States,Washington DC","United States,District of Columbia",1,United States,District of Columbia,,,United States,District of Columbia,,
52936,"United States,District of Columbia,Distrsict o...","United States,District of Columbia",1,United States,District of Columbia,,,United States,District of Columbia,,


In [123]:
df = df[(df['origLocation'] != '') & (df['geoName0'] != '')]
df = df[['origLocation', 'geoName0', 'geoName1', 'geoName2', 'geoName3', 'locationLevels']]
df.to_csv(NEO4J_IMPORT / "10a-GeoLink.csv", index=False)

### Remaining locations that do not match GeoNames.org locations

In [124]:
geo_divisions['canonicalName'] = geo_divisions.apply(lambda x: remove_special_characters(x['name']), axis=1)

In [125]:
geo_divisions.head()

,name,canonicalName,canonicalNameNoAffix
0,Sant Julia de Loria,sant julia de loria,sant julia de loria
1,Ordino,ordino,ordino
2,La Massana,la massana,la massana
3,Encamp,encamp,encamp
4,Canillo,canillo,canillo


In [126]:
divisions = divisions.query("name == ''")

In [127]:
divisions['name'] = divisions.apply(lambda x: get_close_match(x['canonicalName'], geo_divisions['canonicalName']), axis=1)

In [128]:
divisions_close_match = divisions.query("name != ''")

In [129]:
divisions_close_match.shape

(115, 4)

In [130]:
divisions_close_match.head(300)

,origName,canonicalName,canonicalNameNoAffix,name
2,Gomez Palacios,gomez palacios,gomez palacios,gomez palacio
15,Lamine,lamine,lamine,lamin
16,Mehsana,mehsana,mehsana,mesana
17,Barka,barka,barka,barkam
18,Fenoarivo,fenoarivo,fenoarivo,fenoarivobe
21,Al Matariyyah,al matariyyah,matariyyah,al matariyah
45,Manah,manah,manah,matnah
53,Bahla,bahla,bahla,bahula
64,San Francisco PA,san francisco pa,san francisco pa,san francisco putla
76,Zhongxian,zhongxian,zhongxian,zhongxiang


### Locations that do not match

In [131]:
divisions_no_match = divisions.query("name == ''")

In [132]:
divisions_no_match.shape

(405, 4)

In [133]:
divisions_no_match.head(500)

,origName,canonicalName,canonicalNameNoAffix,name
0,Clayton AU,clayton au,clayton au,
1,Gyeonggi,gyeonggi,gyeonggi,
3,Amelia Denis de Icaza,amelia denis de icaza,amelia denis de icaza,
4,Belisario Frías,belisario frias,belisario frias,
5,Vitoria-Gasteiz,vitoria gasteiz,vitoria gasteiz,
6,Omar Torrijos,omar torrijos,omar torrijos,
7,Ernesto Córdoba,ernesto cordoba,ernesto cordoba,
8,Donostia-San Sebastian,donostia san sebastian,donostia san sebastian,
9,Arnulfo Arias,arnulfo arias,arnulfo arias,
11,Contamines,contamines,contamines,
